In [342]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [343]:
data = pd.read_csv("houses.csv")
data.drop(['date', 'street', 'city', 'statezip', 'country'], axis=1, inplace=True)
data = data.loc[data['price'] != 0].reset_index()
data.drop('index', axis=1, inplace=True)
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992


In [344]:
start_ind = 0
end_ind = len(data)

n = round(0.8 * (end_ind - start_ind))

train = data[0:n]
test = data[n:]

In [345]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train['price'].index, y=train['price'], name='train price'))
fig.add_trace(go.Scatter(x=test['price'].index, y=test['price'], name='test price'))

fig.update_layout(title="Выборки",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

In [346]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(start_ind, end_ind)], y=sorted(data['price']), name='train price'))

fig.update_layout(title="Выборки",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

In [347]:
train_x = train.drop('price', axis=1)
train_y = train['price']
test_x = test.drop('price', axis=1)
test_y = test['price']

МНК

In [348]:
from scipy.optimize import curve_fit

In [349]:
def func(curr_data, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13):
  return c1 + c2 * curr_data['bedrooms'] + c3 * curr_data['bathrooms'] + c4 * curr_data['sqft_living'] + \
  c5 * curr_data['sqft_lot'] + c6 * curr_data['floors'] + c7 * curr_data['waterfront'] + c8 * curr_data['view'] + \
  c9 * curr_data['condition'] + c10 * curr_data['sqft_above'] + c11 * curr_data['sqft_basement'] + c12 * curr_data['yr_built'] + \
  c13 * curr_data['yr_renovated']

In [350]:
popt, pcov = curve_fit(func, train_x, train_y)
c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13 = \
popt[0], popt[1], popt[2], popt[3], popt[4], popt[5], popt[6], popt[7], popt[8], popt[9], popt[10], popt[11], popt[12]

res_y_train = c1 + c2 * train['bedrooms'] + c3 * train['bathrooms'] + c4 * train['sqft_living'] + \
  c5 * train['sqft_lot'] + c6 * train['floors'] + c7 * train['waterfront'] + c8 * train['view'] + \
  c9 * train['condition'] + c10 * train['sqft_above'] + c11 * train['sqft_basement'] + c12 * train['yr_built'] + \
  c13 * train['yr_renovated']

res_y_test = c1 + c2 * test['bedrooms'] + c3 * test['bathrooms'] + c4 * test['sqft_living'] + \
  c5 * test['sqft_lot'] + c6 * test['floors'] + c7 * test['waterfront'] + c8 * test['view'] + \
  c9 * test['condition'] + c10 * test['sqft_above'] + c11 * test['sqft_basement'] + c12 * test['yr_built'] + \
  c13 * test['yr_renovated']

res_y = list(res_y_train) + list(res_y_test)

In [351]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train['price'].index, y=train['price'], name='train price'))
fig.add_trace(go.Scatter(x=train['price'].index, y=res_y_train, name='train_МНК price'))

fig.update_layout(title="МНК (train)",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

In [352]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test['price'].index, y=test['price'], name='test price'))
fig.add_trace(go.Scatter(x=test['price'].index, y=res_y_test, name='test_МНК price'))

fig.update_layout(title="МНК (test)",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

In [353]:
sorted_train_data = train.sort_values(by='price').reset_index()
sorted_train_data.drop('index', axis=1, inplace=True)
sorted_train_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,80000.0,1.0,0.75,430,5050,1.0,0,0,2,430,0,1912,0
1,83000.0,2.0,1.00,900,8580,1.0,0,0,3,900,0,1918,0
2,90000.0,2.0,1.00,580,7500,1.0,0,0,3,580,0,1943,2002
3,90000.0,2.0,1.00,790,2640,1.0,0,0,3,790,0,1973,2013
4,100000.0,4.0,1.00,1120,2685,1.0,0,0,3,860,260,1939,1969


In [354]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sorted_train_data['price'].index, y=sorted_train_data['price'], name='train price'))

sorted_res_y_train = sorted([v for v in res_y_train])
fig.add_trace(go.Scatter(x=sorted_train_data['price'].index, y=sorted_res_y_train, name='train_МНК price'))

fig.update_layout(title="МНК (sorted train)",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

In [355]:
sorted_test_data = test.sort_values(by='price').reset_index()
sorted_test_data.drop('index', axis=1, inplace=True)
sorted_test_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,7800.0,2.0,1.0,780,16344,1.0,0,0,1,780,0,1942,0
1,83300.0,3.0,2.0,1490,7770,1.0,0,0,4,1490,0,1990,0
2,83300.0,3.0,2.0,1370,78408,1.0,0,0,5,1370,0,1964,0
3,84350.0,4.0,2.0,2630,16475,2.0,0,0,4,2630,0,1953,1983
4,87500.0,2.0,1.0,780,6685,1.0,0,0,4,780,0,1948,0


In [356]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sorted_test_data['price'].index, y=sorted_test_data['price'], name='test price'))

sorted_res_y_test = sorted([v for v in res_y_test])
fig.add_trace(go.Scatter(x=sorted_test_data['price'].index, y=sorted_res_y_test, name='test_МНК price'))

fig.update_layout(title="МНК (sorted test)",
                  xaxis_title="Номер",
                  yaxis_title="Стоимость")

Регрессия

In [357]:
med = round(sum(data['price']) / len(data)) # средняя цена квартиры

data_regr = [1 if data['price'][i] <= med else 0 for i in range(len(data))]
mnk_regr = [1 if res_y[i] <= med else 0 for i in range(len(res_y))]

In [358]:
from sklearn import metrics

In [359]:
precision = round(metrics.precision_score(data_regr, mnk_regr) * 100, 3)
f1 = round(metrics.f1_score(data_regr, mnk_regr) * 100, 3)
recall = round(metrics.recall_score(data_regr, mnk_regr) * 100, 3)
roc_auc = round(metrics.roc_auc_score(data_regr, mnk_regr) * 100, 3)
accuracy = round(metrics.accuracy_score(data_regr, mnk_regr) * 100, 3)

print("Регрессия")
print("precision:", precision, '%')
print("f1:", f1, '%')
print("recall:", recall, '%')
print("roc_auc:", roc_auc, '%')
print("accuracy:", accuracy, '%')

Регрессия
precision: 86.929 %
f1: 83.977 %
recall: 81.218 %
roc_auc: 79.812 %
accuracy: 80.202 %


In [360]:
fpr, tpr, thresholds = metrics.roc_curve(data_regr, mnk_regr)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, name='roc_curve'))

fig.update_layout(title="ROC curve (Регрессия)",
                  xaxis_title="False positive rate",
                  yaxis_title="True positive rate")

Классификация

In [361]:
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier

In [362]:
data_class = deepcopy(data)
data_class['class'] = [1 if data_class['price'][i] <= med else 0 for i in range(len(data_class))]
data_class.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,class
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,1
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,0
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,1
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,1
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,1


In [363]:
train_class = data_class[0:n]
test_class = data_class[n:]

train_class_x = train_class.drop('class', axis=1)
train_class_y = train_class['class']

test_class_x = test_class.drop('class', axis=1)

In [364]:
classifier = KNeighborsClassifier()
classifier.fit(train_class_x, train_class_y)

res_y_train_class = classifier.predict(train_class_x)
res_y_test_class = classifier.predict(test_class_x)
res_y_class = list(res_y_train_class) + list(res_y_test_class)

In [365]:
precision = round(metrics.precision_score(data_class['class'], res_y_class) * 100, 3)
f1 = round(metrics.f1_score(data_class['class'], res_y_class) * 100, 3)
recall = round(metrics.recall_score(data_class['class'], res_y_class) * 100, 3)
roc_auc = round(metrics.roc_auc_score(data_class['class'], res_y_class) * 100, 3)
accuracy = round(metrics.accuracy_score(data_class['class'], res_y_class) * 100, 3)

print("Классификация")
print("precision:", precision, '%')
print("f1:", f1, '%')
print("recall:", recall, '%')
print("roc_auc:", roc_auc, '%')
print("accuracy:", accuracy, '%')

Классификация
precision: 99.897 %
f1: 99.862 %
recall: 99.828 %
roc_auc: 99.823 %
accuracy: 99.824 %


In [366]:
fpr, tpr, thresholds = metrics.roc_curve(data_class['class'], res_y_class)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, name='roc_curve'))

fig.update_layout(title="ROC curve (Классификация)",
                  xaxis_title="False positive rate",
                  yaxis_title="True positive rate")